# II. **Parsing the tree (PT2)**

### The goal is to correct the bias induced by the casule swap in Klebsiella 
## 1. Information about the Phylo module 
## 2. About the Family 1 ...
Interestingly, the family 1 is constituted of 48__000 prophages. Let's have a closer look into it :
> the fastANI/ Viridic process on those prophages <br>
> The range of prophage size.
A priori, the Family will be dealt with like any other family
## 3. Other processes
***


## 1. About the poackage

some relevant methods 

***
## 2. About the family 1 

***
## 3. Prophage labeling first step 

In [1]:
# On the server 

import os
from collections import Counter
from Bio import Phylo
import Bio
import pandas as pd
from tqdm import tqdm

path_fastani="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_similarity/phageboost/fastANI_20102022_out"
path_label = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_labeling/phageboost/info"
path_ktype="/home/conchae/prediction_depolymerase_tropism"
path_tree="/home/conchae/prediction_depolymerase_tropism/iqtree_local/tree_files/pastML_final_results"

tree = Phylo.read(f"{path_tree}/named.tree_Klensiella_genomes_fixed.2.1.nwk", "newick")

    # Generating the proper cluster file :
    # But first, the strain_ktype dictionary :
strain_ktype={}
good_strain=open(f"{path_ktype}/results_kleborate_count.tsv").read().split("\n")
for index_strain, info in enumerate(good_strain):
    if info:
        strain=info.split("\t")[0].strip()
        ktype=info.split("\t")[2].strip()
        strain_ktype[strain]=ktype
        
        
cluster_df = pd.read_csv(f"{path_fastani}/clusters_99_80.clean.2004.v2.tsv", header = 0, sep="\t")
cluster_dict = cluster_df.to_dict("records")


with open(f"{path_label}/prophage_data.clusters_80.phageboost_70.2504.tsv","w") as outfile :
    outfile.write("Prophage name\tK_serotype_monophyletic_group\tNumber_of_clades\tNumber_of_k_type_swap\tNodes_k_types\n")
    for cluster in tqdm(cluster_dict):
        targets=[element.split("__")[0] for element in cluster["Members"].split(",")]
        LCA=tree.common_ancestor(targets)
        print("That is the LCA \nNow let's welcome the leaders")
        for leader in cluster["Members"].split(",") :
            print("Here is the leader")
            strain_leader=leader.split("__")[0]
            ancestors=LCA.get_path(strain_leader)
            monophylo="wait"
            if len(ancestors)>4:
                for n in range(2,len(ancestors)) :
                    node = ancestors[-n]
                    monophylo = node
                    p_clade=0
                    #Phylo.draw_ascii(node)
                    for clade in node :
                        for target in targets :
                            if target in [terminal.name for terminal in clade.get_terminals()] :
                                p_clade=p_clade+1
                                break
                    if p_clade==1:
                        monophylo = node
                        break
                    else :
                        continue
            else : 
                monophylo=LCA
            k_type_mono=monophylo.comment.split("K_type=")[1].split("'")[0]
            print("The monophylo : ",monophylo, "and it's K-type :", k_type_mono)
            n_ktype_changes=[]
            k_type_changes=[]
            new_ancestors=monophylo.get_path(strain_leader)
            if len(new_ancestors)>1 :
                for old_clade in new_ancestors :
                    k_type_changes.append(old_clade.comment.split("K_type=")[1].split("'")[0])
                print(k_type_changes)
                n_ktype_changes.append(k_type_changes[0])
                print(n_ktype_changes)
                for k_type in k_type_changes :
                    if k_type != n_ktype_changes[-1]:
                        n_ktype_changes.append(old_clade.comment.split("K_type=")[1].split("'")[0])
                        print(n_ktype_changes)
                outfile.write(f"{leader}\t{k_type_mono}\t{monophylo}\t{len(monophylo.get_nonterminals())}\t{len(monophylo.get_terminals())}\t{len(new_ancestors)}\t{len(n_ktype_changes)-1}\t{','.join(n_ktype_changes)}\t{','.join(k_type_changes)}\n")
            else :
                outfile.write(f"{leader}\t{k_type_mono}\t{monophylo}\t{len(monophylo.get_nonterminals())}\t{len(monophylo.get_terminals())}\t{len(new_ancestors)}\t0\tNone\tNone\n")
                
# *****************************************************************************************************************************************************************************************************
#!/bin/bash
#BATCH --job-name=parse_step1_
#SBATCH --partition=medium 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=5
#SBATCH --mem=40gb 
#SBATCH --time=3-00:00:00 
#SBATCH --output=parse_step1_%j.log 

source /storage/apps/ANACONDA/anaconda3/etc/profile.d/conda.sh
conda activate ScaleAP

python /home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_labeling/phageboost/script_files/parse_the_tree.py

FileNotFoundError: [Errno 2] No such file or directory: '/home/conchae/prediction_depolymerase_tropism/iqtree_local/tree_files/pastML_final_results/named.tree_Klensiella_genomes_fixed.2.1.nwk'

***